# Classification d'intention pour Thymio - Notebook complet

Ce notebook contient toutes les étapes pour entraîner un modèle de classification d'intentions basé sur BERT, depuis l'exploration des données jusqu'à la sauvegarde du modèle.

## Étapes couvertes :
1. Chargement et exploration du jeu de données
2. Prétraitement du texte
3. Tokenization avec `BERT`
4. Entraînement du modèle
5. Évaluation sur un jeu de test
6. Sauvegarde du modèle

## 1. Chargement du jeu de données

In [ ]:
import pandas as pd

# Chargement des données
df = pd.read_csv('intent_dataset.csv')
df.head()

## 2. Exploration des données

In [ ]:
# Taille du jeu de données
print(f"Nombre d'exemples : {len(df)}")

# Répartition des classes
df['label'].value_counts().plot(kind='bar', figsize=(12,4), title="Répartition des intentions")

## 3. Prétraitement du texte

In [ ]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^\w\sàâçéèêëîïôûùüÿñæœ'-]", '', text)
    return text

df['text'] = df['text'].apply(clean_text)

## 4. Préparation des données pour BERT

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

# Encodage des étiquettes
le = LabelEncoder()
df['label_id'] = le.fit_transform(df['label'])

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label_id'], test_size=0.2, random_state=42)

# Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(list(X_train), truncation=True, padding=True)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True)

## 5. Dataset Torch

In [ ]:
import torch

class IntentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()} | {'labels': torch.tensor(self.labels[idx])}
    
    def __len__(self):
        return len(self.labels)

train_dataset = IntentDataset(train_encodings, list(y_train))
test_dataset = IntentDataset(test_encodings, list(y_test))

## 6. Entraînement du modèle BERT

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(le.classes_))

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
trainer.train()

## 7. Évaluation

In [ ]:
trainer.evaluate()

## 8. Sauvegarde du modèle et de l’encodeur de labels

In [ ]:
model.save_pretrained('./intent_model')
tokenizer.save_pretrained('./intent_model')

import joblib
joblib.dump(le, './intent_model/label_encoder.pkl')

## Prêt pour l'inférence !